In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
# import tensorflow as tf
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.feature_selection import RFECV
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.ensemble import RandomForestClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("../new_train.csv")
test = pd.read_csv("../new_test.csv")

In [3]:
train

Unnamed: 0                              bidder_id  \
0              0  91a3c57b13234af24875c56fb7e2b2f4rb56a   
1              1  624f258b49e77713fc34034560f93fb3hu3jo   
2              2  1c5f4fc669099bfbfac515cd26997bd12ruaj   
3              3  4bee9aba2abda51bf43d639013d6efe12iycd   
4              4  4ab12bc61c82ddd9c2d65e60555808acqgos1   
...          ...                                    ...   
2008        2008  369515b3af4f8ca582f90271d30b14b6r52aw   
2009        2009  f939c17ffc7c39ac9b35b69e5e75179fv9pe2   
2010        2010  c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl   
2011        2011  0381a69b7a061e9ace2798fd48f1f537mgq57   
2012        2012  84a769adc98498f52debfe57b93a0789556f4   

                            payment_account  \
0     a3d2de7675556553a5f08e4c88d2c228754av   
1     a3d2de7675556553a5f08e4c88d2c228v1sga   
2     a3d2de7675556553a5f08e4c88d2c2280cybl   
3     51d80e233f7b6a7dfdee484a3c120f3b2ita8   
4     a3d2de7675556553a5f08e4c88d2c22857ddh   
...                                     ...   
2008  a1f85275793c4a782f0a668711f41b927ivc9   
2009  a3d2de7675556553a5f08e4c88d2c2286s1m2   
2010  a3d2de7675556553a5f08e4c88d2c22856leq   
2011  fd87037ce0304077079c749f420f0b4c54uo0   
2012  fbe0ce34d6546ebd9e4c63afc68b085byd2tf   

                                    address  outcome  auction  merchandise  \
0     a3d2de7675556553a5f08e4c88d2c228vt0u4      0.0     18.0          1.0   
1     ae87054e5a97a8f840a3991d12611fdcrfbq3      0.0      1.0          1.0   
2     92520288b50f03907041887884ba49c0cl0pd      0.0      4.0          1.0   
3     4cb9717c8ad7e88a9a284989dd79b98dbevyi      0.0      1.0          1.0   
4     2a96c3ce94b3be921e0296097b88b56a7x1ji      0.0     23.0          1.0   
...                                     ...      ...      ...          ...   
2008  e6882cf204a9482edd042b6e31791dfctxzx8      0.0     25.0          1.0   
2009  b9b03d5a127eb07aeb9163cdcf524e1344ac9      0.0      1.0          1.0   
2010  d02c2b288b8aabd79ff47118aff41a2dqwzwc      0.0      1.0          1.0   
2011  f030a221726fbcdfc4dc7dfd1b381a112hieq      0.0      1.0          1.0   
2012  a3d2de7675556553a5f08e4c88d2c228fib6p      0.0      1.0          1.0   

      device   time  country  ...  max_url_per_auction  min_url_per_auction  \
0       14.0   24.0      6.0  ...                  1.0                  1.0   
1        2.0    3.0      1.0  ...                  2.0                  2.0   
2        2.0    4.0      1.0  ...                  1.0                  1.0   
3        1.0    1.0      1.0  ...                  1.0                  1.0   
4       53.0  155.0      2.0  ...                 21.0                  1.0   
...      ...    ...      ...  ...                  ...                  ...   
2008     4.0   33.0      4.0  ...                  1.0                  1.0   
2009     1.0    1.0      1.0  ...                  1.0                  1.0   
2010     2.0    2.0      1.0  ...                  1.0                  1.0   
2011     1.0    1.0      1.0  ...                  1.0                  1.0   
2012     1.0    2.0      1.0  ...                  1.0                  1.0   

      std_url_per_auction  total_no_of_participated_auctions  \
0                0.000000                               18.0   
1                0.000000                                1.0   
2                0.000000                                4.0   
3                0.000000                                1.0   
4                5.644263                               23.0   
...                   ...                                ...   
2008             0.000000                               25.0   
2009             0.000000                                1.0   
2010             0.000000                                1.0   
2011             0.000000                                1.0   
2012             0.000000                                1.0   

      no_of_auction_exceeds_threshold  percentage_of_auctions_above_threshold

In [4]:
train.drop(train.filter(regex="Unname"),axis=1, inplace=True)
test.drop(test.filter(regex="Unname"),axis=1, inplace=True)

In [5]:
X = train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise']) 
y = train['outcome']
X_test_original = test.drop(columns=['bidder_id', 'payment_account', 'address', 'merchandise'])

### Scaling the training and test dataset

In [6]:
X.shape

(2013, 52)

In [7]:
X.columns

Index(['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'auto parts', 'books and music', 'clothing',
       'computers', 'furniture', 'home goods', 'jewelry', 'mobile',
       'office equipment', 'sporting goods', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_excee

In [8]:
scaler = StandardScaler()
# scaled_features = data.copy()
col_names = ['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_exceeds_threshold',
       'percentage_of_auctions_above_threshold', 'total_no_of_bidded_category',
       'no_of_merchandise_exceeds_threshold',
       'percentage_of_merchandise_above_threshold',
       'on_url_that_has_a_bot_mean']

train_features = X[col_names]
scaler = StandardScaler().fit(train_features.values)
train_features = scaler.transform(train_features.values)
X[col_names] = train_features

test_features = X_test_original[col_names]
scaler_test = StandardScaler().fit(test_features.values)
test_features = scaler_test.transform(test_features.values)
X_test_original[col_names] = test_features

In [ ]:
selected_features = ['num_bids_per_ip', 'num_bids_per_auction', 'num_bids_per_country', 'mean_ip_per_auction', 'perc_inst_resp', 'time', 'mean_url_per_auction', 'inst_resp', 'num_bids', 'time_to_bid', 'num_last_bids', 'on_url_that_has_a_bot_mean', 'device', 'percentage_of_auctions_above_threshold', 'num_bids_per_device']
X = X[selected_features]
X_test_original = X_test_original[selected_features]

### Helper Functions

In [9]:
def evaluate(model, X_train, y_train, X_test, y_test):
    print("TRAIN")
    train_predictions = model.predict_proba(X_train)
    train_pred = model.predict(X_train)
        
    train_accuracy = accuracy_score(y_train, train_pred)
    train_auc_roc_score = roc_auc_score(y_train,train_predictions[:,1])
    train_fbeta = fbeta_score(y_train, train_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_train, train_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_train, train_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(train_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(train_auc_roc_score))
    print("*" * 100)
    
    print("TEST")
    
    test_predictions = model.predict_proba(X_test)
    test_pred = model.predict(X_test)
        
    test_accuracy = accuracy_score(y_test, test_pred)
    test_auc_roc_score = roc_auc_score(y_test,test_predictions[:,1])
    test_fbeta = fbeta_score(y_test, test_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_test, test_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_test, test_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(test_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(test_auc_roc_score))
    print("*" * 100)
    
    return [train_accuracy, train_auc_roc_score, train_fbeta, test_accuracy, test_auc_roc_score, test_fbeta]

In [10]:
def process_results(arr):
    train_accuracy = []
    train_auc_roc_score = [] 
    train_fbeta = []
    test_accuracy = []
    test_auc_roc_score = []
    test_fbeta = []
    
    for item in arr:
        train_accuracy.append(item[0])
        train_auc_roc_score.append(item[1])
        train_fbeta.append(item[2])
        test_accuracy.append(item[3])
        test_auc_roc_score.append(item[4])
        test_fbeta.append(item[5])
    
    mean_accuracy = np.array(train_accuracy).mean()
    mean_train_auc_roc_score = np.array(train_auc_roc_score).mean()
    mean_train_fbeta = np.array(train_fbeta).mean()
    mean_test_accuracy = np.array(test_accuracy).mean()
    mean_test_auc_roc_score = np.array(test_auc_roc_score).mean()
    mean_test_fbeta = np.array(test_fbeta).mean()
    
    print("final train accuracy: " + str(mean_accuracy))
    print("final train AUC: " + str(mean_train_auc_roc_score))
    print("final train fbeta: " + str(mean_train_fbeta))
    print("final test accuracy: " + str(mean_test_accuracy))
    print("final test AUC: " + str(mean_test_auc_roc_score))
    print("final test fbeta: " + str(mean_test_fbeta))

### SKFold + SMOTE (Base Model)

In [11]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
counter = 1
ann_result = []

over = SMOTE(sampling_strategy=0.2, random_state = 42)
under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

for train_index, test_index in skf.split(X, y):
    print("Fold number " + str(counter))
    counter += 1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    # ann
    print("ANN")
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann.fit(X_train, y_train)
    ann_result.append(evaluate(ann, X_train, y_train, X_test, y_test))

Fold number 1
DECISION TREE
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       381
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000       686
   macro avg     1.0000    1.0000    1.0000       686
weighted avg     1.0000    1.0000    1.0000       686

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9681    0.8743    0.9188       382
         1.0     0.1724    0.4762    0.2532        21

    accuracy                         0.8536       403
   macro avg     0.5703    0.6753    0.5860       403
weighted avg     0.9267    0.8536    0.8842       403

FBeta Score
0.352112676056338
Model Performance
Accuracy = 0.8536%.
AUC ROC = 0.6753%.
**

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       381
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000       686
   macro avg     1.0000    1.0000    1.0000       686
weighted avg     1.0000    1.0000    1.0000       686

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9721    0.9136    0.9420       382
         1.0     0.2500    0.5238    0.3385        21

    accuracy                         0.8933       403
   macro avg     0.6111    0.7187    0.6402       403
weighted avg     0.9345    0.8933    0.9105       403

FBeta Score
0.4296875
Model Performance
Accuracy = 0.8933%.
AUC ROC = 0.8839%.
**************************************

C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.pred

Classification report
              precision    recall  f1-score   support

         0.0     0.9859    0.3665    0.5344       382
         1.0     0.0728    0.9048    0.1348        21

    accuracy                         0.3945       403
   macro avg     0.5294    0.6356    0.3346       403
weighted avg     0.9383    0.3945    0.5135       403

FBeta Score
0.2753623188405797
Model Performance
Accuracy = 0.3945%.
AUC ROC = 0.7662%.
****************************************************************************************************
Fold number 2
DECISION TREE
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       381
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000       686
   macro avg     1.0000    1.0000    1.0000       686
weighted avg     1.0000    1.0000    1.0000       686

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
******

...

Classification report
              precision    recall  f1-score   support

         0.0     0.9773    0.6754    0.7988       382
         1.0     0.1014    0.7000    0.1772        20

    accuracy                         0.6766       402
   macro avg     0.5394    0.6877    0.4880       402
weighted avg     0.9337    0.6766    0.7678       402

FBeta Score
0.3211009174311927
Model Performance
Accuracy = 0.6766%.
AUC ROC = 0.7513%.
****************************************************************************************************
Fold number 5
DECISION TREE
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       381
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000       686
   macro avg     1.0000    1.0000    1.0000       686
weighted avg     1.0000    1.0000    1.0000       686

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
******

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       381
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000       686
   macro avg     1.0000    1.0000    1.0000       686
weighted avg     1.0000    1.0000    1.0000       686

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9771    0.8927    0.9330       382
         1.0     0.2264    0.6000    0.3288        20

    accuracy                         0.8781       402
   macro avg     0.6017    0.7463    0.6309       402
weighted avg     0.9397    0.8781    0.9029       402

FBeta Score
0.4511278195488722
Model Performance
Accuracy = 0.8781%.
AUC ROC = 0.8912%.
********************************************************

C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.pred

Classification report
              precision    recall  f1-score   support

         0.0     0.9545    0.8796    0.9155       382
         1.0     0.0800    0.2000    0.1143        20

    accuracy                         0.8458       402
   macro avg     0.5173    0.5398    0.5149       402
weighted avg     0.9110    0.8458    0.8757       402

FBeta Score
0.15384615384615388
Model Performance
Accuracy = 0.8458%.
AUC ROC = 0.2676%.
****************************************************************************************************


In [15]:
process_results(ann_result)

final train accuracy: 0.6346938775510205
final train AUC: 0.5782892302396626
final train fbeta: 0.552316706177311
final test accuracy: 0.6980259990370727
final test AUC: 0.6061761406133134
final test fbeta: 0.2935796319459635


### SKFold + SMOTE (Random Search)

In [34]:
def random_search_ann(X, y):
    print("RANDOM SEARCH ANN EXPERIMENT")
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]

    ann_random_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)

    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_random = RandomizedSearchCV(estimator = ann, param_distributions = ann_random_grid, n_iter = 100, cv = skf, 
                                verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    ann_random.fit(X_train, y_train)
    ann_best_random = ann_random.best_estimator_
    ann_random_accuracy = evaluate(ann_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for ANN")
    print(ann_random.best_params_)

In [34]:
random_search_ann(X, y)

RANDOM SEARCH ANN EXPERIMENT
Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.


*****************************************************************************************
## Best Random Search Param for ANN
{}

### SKFold + SMOTE (Grid Search)

In [38]:
def grid_search_ann(X, y):
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]
    
    ann_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_grid = GridSearchCV(estimator = ann, param_grid = ann_grid, n_jobs=-1, cv=skf, scoring='roc_auc')
    ann_grid.fit(X_train, y_train)
    
    ann_best_grid = ann_grid.best_estimator_
    ann_grid_accuracy = evaluate(ann_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for ANN")
    print(ann_grid.best_params_)
    return ann_best_grid

In [ ]:
ann_best_grid = grid_search_ann(X, y)